<a href="https://colab.research.google.com/github/yakimsanik/belhard/blob/master/HW4/Yakim_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
pip install catboost

In [41]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from catboost import CatBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [12]:
# Загрузка датасета
shark_attacks = pd.read_csv('/content/drive/MyDrive/Project/Data Science/HW4/titanic.csv', encoding='latin1')

In [14]:
df = sns.load_dataset('titanic')
df.head(4)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False


In [15]:
# Удаление строк с пропусками в целевом признаке и выборе фичей
df = df.dropna(subset=['survived', 'age', 'fare'])
features = ['pclass', 'age', 'fare', 'sex', 'sibsp', 'parch']
X = df[features].copy()
y = df['survived']

In [16]:
# Преобразование категориального признака 'sex' в числовой (male=1, female=0)
X['sex'] = X['sex'].map({'male': 1, 'female': 0})

In [17]:
# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:
# Масштабирование числовых признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Обучение модели без балансировки классов (Baseline)

In [34]:
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

print("=== Baseline Model (No Balancing) ===")
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, model.predict_proba(X_test_scaled)[:, 1]))

=== Baseline Model (No Balancing) ===
              precision    recall  f1-score   support

           0       0.78      0.85      0.81       126
           1       0.76      0.66      0.71        89

    accuracy                           0.77       215
   macro avg       0.77      0.76      0.76       215
weighted avg       0.77      0.77      0.77       215

ROC-AUC: 0.8261548064918851


Применение Oversampling (RandomOverSampler)

In [39]:
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train_scaled, y_train)

model_ros = LogisticRegression(random_state=42)
model_ros.fit(X_train_ros, y_train_ros)
y_pred_ros = model_ros.predict(X_test_scaled)

print("\n=== Oversampling (RandomOverSampler) ===")
print(classification_report(y_test, y_pred_ros))
print("ROC-AUC:", roc_auc_score(y_test, model_ros.predict_proba(X_test_scaled)[:, 1]))


=== Oversampling (RandomOverSampler) ===
              precision    recall  f1-score   support

           0       0.80      0.78      0.79       126
           1       0.70      0.72      0.71        89

    accuracy                           0.75       215
   macro avg       0.75      0.75      0.75       215
weighted avg       0.75      0.75      0.75       215

ROC-AUC: 0.8273140716960942


Применение Undersampling (RandomUnderSampler)

In [42]:
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train_scaled, y_train)

model_rus = LogisticRegression(random_state=42)
model_rus.fit(X_train_rus, y_train_rus)
y_pred_rus = model_rus.predict(X_test_scaled)

print("\n=== Undersampling (RandomUnderSampler) ===")
print(classification_report(y_test, y_pred_rus))
print("ROC-AUC:", roc_auc_score(y_test, model_rus.predict_proba(X_test_scaled)[:, 1]))


=== Undersampling (RandomUnderSampler) ===
              precision    recall  f1-score   support

           0       0.80      0.77      0.79       126
           1       0.69      0.73      0.71        89

    accuracy                           0.75       215
   macro avg       0.75      0.75      0.75       215
weighted avg       0.76      0.75      0.75       215

ROC-AUC: 0.8284733369003031


In [23]:
# Инициализация моделей
models = {
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "Extra Trees": ExtraTreesClassifier(random_state=42),
    "QDA": QuadraticDiscriminantAnalysis(),
    "LightGBM": LGBMClassifier(random_state=42),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric='logloss'),
    "Dummy": DummyClassifier(strategy='most_frequent'),
    "SVM Linear": SVC(kernel='linear', probability=True, random_state=42)
}

In [24]:
# Обучение и оценка моделей
results = []
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]

    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1-score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_proba)
    })

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 201, number of negative: 298
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000897 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 179
[LightGBM] [Info] Number of data points in the train set: 499, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.402806 -> initscore=-0.393789
[LightGBM] [Info] Start training from score -0.393789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [25]:
# Создание таблицы результатов
results_df = pd.DataFrame(results).sort_values('ROC-AUC', ascending=False)
print(results_df)

                Model  Accuracy  F1-score   ROC-AUC
8             XGBoost  0.809302  0.757396  0.864232
5            LightGBM  0.800000  0.748538  0.861468
0   Gradient Boosting  0.795349  0.741176  0.858837
1            CatBoost  0.786047  0.729412  0.851659
4                 QDA  0.776744  0.717647  0.834091
6                 KNN  0.795349  0.747126  0.831594
2            AdaBoost  0.767442  0.712644  0.830257
3         Extra Trees  0.776744  0.710843  0.811842
10         SVM Linear  0.753488  0.690058  0.799670
7       Decision Tree  0.762791  0.711864  0.759230
9               Dummy  0.586047  0.000000  0.500000


In [26]:
# Вывод лучшей модели
best_model_name = results_df.iloc[0]['Model']
print(f"\nЛучшая модель: {best_model_name}")


Лучшая модель: XGBoost


In [27]:
# Детальный отчет лучшей модели
best_model = models[best_model_name]
print("\nДетальный отчет классификации:")


Детальный отчет классификации:


In [28]:
best_model = models[best_model_name]
y_pred = best_model.predict(X_test_scaled)
y_proba = best_model.predict_proba(X_test_scaled)[:, 1]

print(f"\n=== Детальный отчет для лучшей модели ({best_model_name}) ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_proba):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted']))


=== Детальный отчет для лучшей модели (XGBoost) ===
Accuracy: 0.8093
F1-score: 0.7574
ROC-AUC: 0.8642

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       126
           1       0.80      0.72      0.76        89

    accuracy                           0.81       215
   macro avg       0.81      0.80      0.80       215
weighted avg       0.81      0.81      0.81       215

Confusion Matrix:
Predicted    0   1
Actual            
0          110  16
1           25  64
